In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']= '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import pretty_midi as pyd
from arrangement_utils import *
import warnings
warnings.filterwarnings("ignore")

"""Download checkpoints and data at https://we.tl/t-cc2nOC8dAA (379MB) and decompress at the directory"""
DATA_FILE_ROOT = './data_file_dir/'
DEVICE = 'cuda:0'
piano_arranger, orchestrator, piano_texture, band_prompt = load_premise(DATA_FILE_ROOT, DEVICE)

### Initialize input and preference
We provide three sample lead sheets for a quick inference. You should be able to directly run the code blocks after downloading the pre-trained checkpoints.

If you wish to test our model on your own lead sheet file, please initialize a sub-folder with its `SONG_NAME` in the `./demo` folder and put the file in, and name the file "lead sheet.mid". 

Please also specify `SEGMENTATION` (phrase structure) and `NOTE_SHIFT` (the duration of the pick-up measure if any).

In [5]:
"""Set input lead sheet"""
SONG_NAME, SEGMENTATION, NOTE_SHIFT = 'Castles in the Air', 'A8A8B8B8', 1   #1 beat in the pick-up measure
#SONG_NAME, SEGMENTATION, NOTE_SHIFT = 'Jingle Bells', 'A8B8A8', 0
#SONG_NAME, SEGMENTATION, NOTE_SHIFT = 'Sally Garden', 'A4A4B4A4', 0

"""Set texture pre-filtering for piano arrangement (default random)"""
RHTHM_DENSITY = np.random.randint(3, 5)
VOICE_NUMBER = np.random.randint(3, 5)
PREFILTER = (RHTHM_DENSITY, VOICE_NUMBER)

"""Set if use a 2-bar prompt for full-band arrangement (default False)""" 
USE_PROMPT = False

lead_sheet = read_lead_sheet('./demo', SONG_NAME, SEGMENTATION, NOTE_SHIFT)

### Piano Accompaniment Arrangement

In [6]:
midi_piano, acc_piano = piano_arrangement(*lead_sheet, *piano_texture, piano_arranger, PREFILTER)
midi_piano.write(f'./demo/{SONG_NAME}/arrangement_piano.mid')

Phrasal Unit selection begins:
	 4 phrases in the lead sheet;
	 set note density filter: (4, 4).


100%|██████████| 3/3 [00:17<00:00,  5.82s/it]


Re-harmonization begins ...
Piano accompaiment generated!


### Orchestration

In [7]:
midi_prompt, func_prompt = prompt_sampling(acc_piano, *band_prompt, DEVICE)
if USE_PROMPT:
    midi_band = orchestration(acc_piano, None, *func_prompt, orchestrator, DEVICE, blur=.5, p=.1, t=4)
else:
    instruments, pitch_prompt, time_promt = func_prompt
    midi_band = orchestration(acc_piano, None, instruments, None, None, orchestrator, DEVICE, blur=.5, p=.1, t=4)
mel_track = pyd.Instrument(program=72, is_drum=False, name='melody')
mel_track.notes = midi_piano.instruments[0].notes
midi_band.instruments.append(mel_track)
midi_band.write(f'./demo/{SONG_NAME}/arrangement_band.mid')

Prior model initialized with 7 tracks:
	['Acoustic Bass', 'Chromatic Percussion', 'Synth Pad', 'Acoustic Piano', 'Electric Piano', 'Acoustic Bass', 'Acoustic Guitar']
Orchestration begins ...
Full-band accompaiment generated!
